In [1]:
##denpendents 
import os
import shutil
import random

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import shutil
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.metrics import recall_score, precision_score, classification_report, accuracy_score
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy, categorical_crossentropy

In [10]:
from google.colab import drive
drive.mount('/content/drive')

RAW_DIR1 = "/content/drive/MyDrive/data/raw"


KeyboardInterrupt: 

In [2]:
##folder structures and directory
RAW_DIR = "data/raw"
OUT_DIR = "data"
BAD_DIR = "data/corrupt"

##image resize process
IMG_HEIGHT=180    ##Resize all images
IMG_WIDTH=180   ##Resize all images
CHANNELS=3
BATCH_SIZE=32 
EPOCHS=10

##splitting ratio
TRAIN_RATIO = 0.70
VAL_RATIO = 0.15
TEST_RATIO = 0.15

##Reproducibility
SEED = 42
random.seed(SEED)

In [3]:
##Instead of deleting immediately, we can move it
#Move corrupted images to a separate folder 

def move_corrupted_images(image_dir, bad_dir):
    os.makedirs(bad_dir, exist_ok=True)

    for root, _, files in os.walk(image_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                path = os.path.join(root, file)
                try:
                    img = Image.open(path)
                    img.verify()
                except:
                    shutil.move(path, os.path.join(bad_dir, file))

In [4]:
##move the corrupted files
move_corrupted_images(RAW_DIR,BAD_DIR)

## Data Loading using Tensorflow

In [ ]:
##data loading 
raw_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    "",
    shuffle=True,
    image_size=(IMG_HEIGHT,IMG_WIDTH),
    batch_size=BATCH_SIZE
)

NotFoundError: Could not find directory /Users/sarojrai/Desktop/Computer Vision/Plant Diseases Detection/data/raw

In [ ]:
##class name
class_name=raw_dataset.class_names
len(class_name)
class_name

<h2 style='text-align:center; font-family:Geogeria; color:white'><b>Hardcode: Splitting Raw Data</b></h2>

In [ ]:
# ##Identify class labels
# folder_name_list=os.listdir(RAW_DIR)

# ##Folder names become class labels
# ##keeps only folders ["apple", "mango", "grapes", "potato"]
# classes = [cls for cls in folder_name_list if os.path.isdir(os.path.join(RAW_DIR, cls))] 

# # Create directory structure
# ##Create output directory structure
# ##Iterates over dataset splits
# for split in ["train", "val", "test"]:
#     ##Iterates over class labels
#     for clss in classes:
#         os.makedirs(os.path.join(OUT_DIR, split, clss), exist_ok=True) #Creates directories

# # Split per class
# #Process one class at a time
# #This ensures class-wise stratification
# for cls in classes:
#     #full path to one class folder like data/raw/apple
#     cls_path = os.path.join(RAW_DIR, cls)
#     ##list of images within list of folder 
#     img_list=os.listdir(cls_path)
#     #Collects only image files
#     images = [f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))]

#     ##image suffling
#     random.shuffle(images)

#     #Compute split sizes
#     n_total = len(images)
#     n_train = int(TRAIN_RATIO * n_total)
#     n_val = int(VAL_RATIO * n_total)

#     train_imgs = images[:n_train]
#     val_imgs = images[n_train:n_train + n_val]
#     test_imgs = images[n_train + n_val:]

#     #Copy images into split folders
#     for img in train_imgs:
#         shutil.copy(
#             os.path.join(cls_path, img),
#             os.path.join(OUT_DIR, "train", cls, img)
#         )

#     for img in val_imgs:
#         shutil.copy(
#             os.path.join(cls_path, img),
#             os.path.join(OUT_DIR, "val", cls, img)
#         )

#     for img in test_imgs:
#         shutil.copy(
#             os.path.join(cls_path, img),
#             os.path.join(OUT_DIR, "test", cls, img)
#         )

# print("Train / Validation / Test split completed.")

<h2 style='text-align:center; font-family:Geogeria; color:white'><b>Data Preprocessing: Converting images into an Array</b></h2>

In [ ]:
# ##data directories
# TRAIN_DIR='data/train'
# TEST_DIR='data/test'
# VAL_DIR='data/val'

# # Loads images from directories, automatically assigns labels from folder names
# ##output dataset format look like this (images,labels) in memory
# train_ds = tf.keras.utils.image_dataset_from_directory( 
#     TRAIN_DIR,
#     image_size=(IMG_HIGHT,IMG_WIDTH),
#     batch_size=BATCH_SIZE,
#     shuffle=True,       # Shuffle training dataset
#     seed=42,
#     validation_split=False
# )

# ##validation data preprocess
# val_ds = tf.keras.utils.image_dataset_from_directory( 
#     VAL_DIR,
#     image_size=(IMG_HIGHT,IMG_WIDTH),
#     batch_size=BATCH_SIZE,
#     shuffle=False,       #shuffle is not needed for validation/test
#     validation_split=False
# )

# ##test data preprocess
# test_ds = tf.keras.utils.image_dataset_from_directory(
#     TEST_DIR,
#     image_size=(IMG_HIGHT,IMG_WIDTH),
#     batch_size=BATCH_SIZE,
#     shuffle=False,    #shuffle is not needed for validation/test
#     validation_split=False
# )

In [ ]:
# ##disease class
# dis_class=train_ds.class_names

In [ ]:
# for x, y in train_ds.take(1):
#     print(type(x), x.shape)


## Alternative way to split data

In [ ]:
len(raw_dataset)*TRAIN_RATIO

In [ ]:
train=raw_dataset.take(550)
len(train)

In [ ]:
rest_data=raw_dataset.skip(550)
len(rest_data)

In [ ]:
len(raw_dataset)*VAL_RATIO

In [ ]:
len(raw_dataset)*TEST_RATIO

In [ ]:
test=rest_data.take(118)
len(test)

In [ ]:
val=rest_data.skip(118)
len(val)

In [ ]:
##user define function to perform data spliting
def train_test_partition(data, train_split=0.80, test_split=0.1, val_split=0.1, shuffle=True, shuffle_size=100):
   total_data= len(data)

   if shuffle:
      data = data.shuffle(shuffle_size, SEED)
    
   ## train, test, validation size based on the given data and splitting size
   train_size=int(total_data*train_split)
   val_size=int(total_data*val_split)
   
   ##splitted data
   train_set=data.take(train_size)
   val_set=data.skip(train_size).take(val_size)
   test_set=data.skip(train_size).skip(val_size)

   ## return the train, test and validation set
   return train_set, test_set, val_set

In [ ]:
##calling the function to extract train, test, validation data 
train_data, test_data, val_data = train_test_partition(raw_dataset)

In [ ]:
##checking 
len(train_data)

<h2 style='text-align:center; font-family:Geogeria; color:white'><b>Display Images</b></h2>

In [ ]:
##print images from the train dataset
## taking 1st batch only take(1)

plt.figure(figsize=(10,10))
for image,labels in raw_dataset.take(1):
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        plt.imshow(np.array(image[i]).astype('uint8'))
        plt.title(class_name[labels[i]])
        plt.axis("off")
plt.show()

In [ ]:
## number of class
##class name
class_name=raw_dataset.class_names
n_classes=len(class_name)

In [ ]:
# train_ds.cardinality().numpy()

In [ ]:
## model building
##image input size
input_shapes=(BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, CHANNELS)

##my model architecture
model= Sequential([
    ##image resizing
    layers.Resizing(IMG_HEIGHT,IMG_WIDTH),
    ##normalisation layer
    layers.Rescaling(1./255),
    ##randomly flip the images
    layers.RandomFlip("horizontal_and_vertical"),
    ##random rotation
    layers.RandomRotation(0.2),

    ##first convolutional layer
    layers.Conv2D(16, (3,3), activation='relu', padding='same', input_shape=input_shapes),
    ##pooling layer
    layers.MaxPooling2D(),
    ##2nd convolutional layer
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(),

    ##3rd convolutional layer
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(),

    ##convert into a vector
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    
    ##output layer
    layers.Dense(n_classes, activation='softmax')
])

#the model parameters, use the .build() method to see trainable parameters otherwise it won't be needed
model.build(input_shape=input_shapes)

In [ ]:
##helps to check trainable parameter
model.summary()

In [ ]:
##compile the model with optimizers
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
##training the model
model.fit(train_data,
          epochs=EPOCHS,
          verbose=1,
          batch_size=BATCH_SIZE,
          validation_data=val_data)